<a href="https://colab.research.google.com/github/yeonkkk/AIFFEL-Project/blob/main/Exploration9/project/%5BE_09%5DMovie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E-09. Movie Recommendation

## 데이터 탐색 및 전처리

- 사용할 데이터: [MovieLens 1M Dataset](https://grouplens.org/datasets/movielens/1m/)

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있음

- 별점을 시청횟수로 해석할 것

- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외할 것

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

rating_file_path = "/content/drive/MyDrive/ex9/ratings.dat"
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::',
                      names=ratings_cols, 
                      engine='python', 
                      encoding = "ISO-8859-1")

- 불러온 데이터 확인하기

- ratings 컬럼
  - user_id: 사용자별 고유 번호
  - movie_id: 영화별 고유 번호
  - ratings: 사용자가 남긴 평점 (1~5)
  - timestamp: 날짜와 시간


In [ ]:
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


- 평점이 3점 미만인 경우 선호하지 않기로 하였으니, 3점 이상인 영화들만 남긴다.

- 결과적으로 전체의 83% 정도의 영화만 활용

In [ ]:
# 3점 이상만 남기기
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)




print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [ ]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
# 별점을 재생 횟수로 생각하기로 했으므로!
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [ ]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

- 사용하지 않을 컬럼은 삭제해 준다.

In [ ]:
del ratings['timestamp']

In [ ]:
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


- 영화 이름과 장르를 확인할 수 있는 파일인 movies 을 불러온다.

In [ ]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path= "/content/drive/MyDrive/ex9/movies.dat"
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# 검색을 쉽게 하기 위해 문자열을 소문자로 바꿔줍시다.
movies['title'] = movies['title'].str.lower() 
movies.head(5)

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy



- 유저수, 영화수, 인기 많은 영화 확인

- 유저들이 몇 개의 영화를 보고 있는지에 대한 통계

- 유저 영화 시청 횟수 중앙값에 대한 통계

In [ ]:
# 유저 수
ratings['user_id'].nunique()

6039

In [ ]:
# 영화 수
ratings['movie_id'].nunique()

3628

In [ ]:
len(ratings['user_id'])

836478

- 활용할 컬럼만 남기고 나머지 컬럼은 삭제한다.

In [ ]:
del movies['genre']

- 활용 및 가시적인 편의성을 위해 두 dataframe을 합친다

In [ ]:
data = pd.merge(ratings, movies, on='movie_id')
data = data[['user_id', 'movie_id', 'title', 'counts']]
data

,user_id,movie_id,title,counts
0,1,1193,one flew over the cuckoo's nest (1975),5
1,2,1193,one flew over the cuckoo's nest (1975),5
2,12,1193,one flew over the cuckoo's nest (1975),4
3,15,1193,one flew over the cuckoo's nest (1975),4
4,17,1193,one flew over the cuckoo's nest (1975),5
...,...,...,...,...
836473,5851,3607,one little indian (1973),5
836474,5854,3026,slaughterhouse (1987),4
836475,5854,690,"promise, the (versprechen, das) (1994)",3
836476,5938,2909,"five wives, three secretaries and me (1998)",4


- 인기 영화 30개 확인해보기 

In [ ]:
# 인기 많은 영화
movie_count = data.groupby(['movie_id', 'title'])['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id  title                                                
2858      american beauty (1999)                                   3211
260       star wars: episode iv - a new hope (1977)                2910
1196      star wars: episode v - the empire strikes back (1980)    2885
1210      star wars: episode vi - return of the jedi (1983)        2716
2028      saving private ryan (1998)                               2561
589       terminator 2: judgment day (1991)                        2509
593       silence of the lambs, the (1991)                         2498
1198      raiders of the lost ark (1981)                           2473
1270      back to the future (1985)                                2460
2571      matrix, the (1999)                                       2434
480       jurassic park (1993)                                     2413
2762      sixth sense, the (1999)                                  2385
608       fargo (1996)                                             2371


In [ ]:
# 유저별 몇 개의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [ ]:
# 유저별 counts 중앙값에 대한 통계
user_median = data.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

- 이후 영화 추천을 위해 새로운 사용자의 선호 영화 추가

- 5개의 영화를 추가해보았으며, 각각 5, 4, 3, 5, 5회 시청하였다고 가정하였다.

In [ ]:
# 좋아하는 영화 5개 선택하기
my_movie_id = ['1036', '2987', '1270', '1259', '1196']
my_favorite = ['die hard (1988)', 
               'who framed roger rabbit? (1988)', 
               'back to the future (1985)',
              'stand by me (1986)',
              'star wars: episode v - the empire strikes back (1980)']

# '6100'이라는 user_id가 각각의 횟수만큼 영화를 봤다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['6100']*5,
                             'movie_id': my_movie_id,
                             'title': my_favorite,
                             'counts':[5, 4, 3, 5, 5]})

if not data.isin({'user_id':['6100']})['user_id'].any():  # user_id에 '6100'이라는 데이터가 없다면
    data = data.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,title,counts
836473,5851,3607,one little indian (1973),5
836474,5854,3026,slaughterhouse (1987),4
836475,5854,690,"promise, the (versprechen, das) (1994)",3
836476,5938,2909,"five wives, three secretaries and me (1998)",4
836477,5948,1360,identification of a woman (identificazione di ...,5
0,6100,1036,die hard (1988),5
1,6100,2987,who framed roger rabbit? (1988),4
2,6100,1270,back to the future (1985),3
3,6100,1259,stand by me (1986),5
4,6100,1196,star wars: episode v - the empire strikes back...,5


## CSR matrix 만들기

In [ ]:
# 인덱싱 하기 
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [ ]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['6100']) 
print(movie_to_idx['die hard (1988)'])

6039
194


In [ ]:
data['user_id'].nunique()

6040

In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id2'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id2'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,title,counts,user_id2,movie_id2
0,1,1193,one flew over the cuckoo's nest (1975),5,0,0
1,2,1193,one flew over the cuckoo's nest (1975),5,1,0
2,12,1193,one flew over the cuckoo's nest (1975),4,2,0
3,15,1193,one flew over the cuckoo's nest (1975),4,3,0
4,17,1193,one flew over the cuckoo's nest (1975),5,4,0
...,...,...,...,...,...,...
0,6100,1036,die hard (1988),5,6039,194
1,6100,2987,who framed roger rabbit? (1988),4,6039,201
2,6100,1270,back to the future (1985),3,6039,22
3,6100,1259,stand by me (1986),5,6039,80


In [ ]:
from scipy.sparse import csr_matrix

num_user = data['user_id2'].nunique()
num_movie = data['movie_id2'].nunique()

csr_data = csr_matrix((data.counts, 
                       (data.user_id2, data.movie_id2)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 모델 설계, 훈련

In [ ]:
pip install implicit

     |████████████████████████████████| 1.1 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606537 sha256=860f02e1c099dee0a82ec0faeb98fc8db020bad89263a45f6be6c1eaced03793
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.01, 
                                    use_gpu=False, 
                                    iterations=15, 
                                    dtype=np.float32)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [ ]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 예측 확인하기

### 새로운 영화에 대한 선호도 예측

In [ ]:
# 6100 벡터와 stand by me의 벡터를 어떻게 만들고 있는지 살펴보기
yeonk, stand_by_me = user_to_idx['6100'], movie_to_idx['stand by me (1986)']
yeonk_vector, stand_by_me_vector = als_model.user_factors[yeonk], als_model.item_factors[stand_by_me]

In [ ]:
yeonk_vector

array([-0.13309902, -0.14665395,  0.23551907, -0.25294033,  0.57409096,
        0.5231553 , -0.5521547 ,  0.14402701,  0.41556826, -0.10951859,
        0.09305875,  0.11173218, -0.1920309 , -0.09276097,  0.09995529,
        0.02002639,  0.65449923,  0.39605168, -0.74272746, -0.34615666,
        0.50725096,  1.0886774 ,  0.6992015 , -0.31426752, -0.12645455,
        0.4181726 ,  0.72847164,  0.3257795 , -0.13918799,  0.11838039,
       -0.996633  ,  0.20965172, -0.31082174,  0.21506648, -0.11336543,
       -0.21306463, -0.60583043,  0.14916945,  0.52444005,  0.10753248,
        0.80157316, -0.9764798 , -0.01998127, -0.16507287, -0.39112118,
        0.1893841 , -0.9918707 , -0.3256325 ,  0.27220958,  0.12936632,
        0.16456783, -1.3457015 ,  0.5635626 ,  0.22574285, -0.28947318,
       -0.24534397,  0.77191263,  0.25438341,  0.7557651 ,  0.5303375 ,
        0.09244028,  0.53696823, -0.4031666 ,  0.87905496,  1.0348989 ,
       -0.78435355,  0.5678662 ,  0.4272789 ,  0.46423224, -0.85

In [ ]:
stand_by_me_vector

array([ 0.00766326,  0.0009511 ,  0.0086639 ,  0.01795044,  0.02529761,
       -0.00568419, -0.00738854,  0.03047804,  0.0169243 , -0.00243875,
       -0.01099014,  0.01476682,  0.01886408,  0.00242149,  0.00396036,
       -0.00513673, -0.00632041, -0.00862189, -0.01187225,  0.03256232,
        0.00787817,  0.03420976,  0.04531543, -0.0027176 , -0.01515944,
       -0.00292947,  0.02646234,  0.01153296,  0.00453851, -0.01659384,
       -0.0018543 ,  0.00105652,  0.00772733,  0.00423437, -0.01411476,
       -0.0140371 , -0.02572809,  0.02818452,  0.03850204,  0.02004455,
       -0.01704827,  0.00363544,  0.02075369,  0.01472506,  0.0030869 ,
       -0.00403751, -0.01317057, -0.01084308, -0.01258986,  0.00588472,
       -0.00472328, -0.02333257,  0.06069714, -0.00997492, -0.00153336,
       -0.00873834, -0.01330642,  0.01734079,  0.03214543,  0.02862642,
        0.01268846, -0.00623462,  0.00947961,  0.01801164,  0.0483378 ,
       -0.00676408,  0.01722723,  0.0168564 ,  0.02233932, -0.00

In [ ]:
# 6100과 stand by me를 내적하는 코드
np.dot(yeonk_vector, stand_by_me_vector)

0.5385043

- 선호 영화 목록에 없었던 toy story로 선호도 확인해보기

In [ ]:
# toy story에 대한 선호도 예측 확인
toy_story = movie_to_idx['toy story (1995)']
toy_story_vector = als_model.item_factors[toy_story]
np.dot(yeonk_vector, toy_story_vector)

0.120279804

- 선호 영화 목록 중 5회 시청했다고 가정했던 star wars5에 대한 선호도 확인해보기

In [ ]:
# star wars5에 대한 선호도 예측 확인
star_wars5 = movie_to_idx['star wars: episode v - the empire strikes back (1980)']
star_wars5_vector = als_model.item_factors[star_wars5]
np.dot(yeonk_vector, star_wars5_vector)

0.5869175

### 비슷한 영화 찾아보기

In [ ]:
# star_wars5 와 비슷한 영화 찾아보기
favorite_movie = 'star wars: episode v - the empire strikes back (1980)'
movie_id2 = movie_to_idx[favorite_movie]
similar_movie= als_model.similar_items(movie_id2, N=15)
similar_movie

[(117, 1.0000001),
 (64, 0.8933432),
 (44, 0.8703181),
 (120, 0.68372875),
 (200, 0.48182806),
 (22, 0.45815232),
 (172, 0.45219558),
 (651, 0.42305163),
 (5, 0.40833947),
 (60, 0.3761487),
 (124, 0.37106925),
 (193, 0.36854777),
 (26, 0.36700907),
 (680, 0.3496446),
 (550, 0.31380063)]

In [ ]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['star wars: episode v - the empire strikes back (1980)',
 'star wars: episode vi - return of the jedi (1983)',
 'star wars: episode iv - a new hope (1977)',
 'raiders of the lost ark (1981)',
 'terminator, the (1984)',
 'back to the future (1985)',
 'indiana jones and the last crusade (1989)',
 'aliens (1986)',
 'princess bride, the (1987)',
 'star wars: episode i - the phantom menace (1999)',
 'matrix, the (1999)',
 'alien (1979)',
 'e.t. the extra-terrestrial (1982)',
 'blade runner (1982)',
 'dr. strangelove or: how i learned to stop worrying and love the bomb (1963)']

In [ ]:
# 위 과정을 함수로 만들기
def get_similar_movie(movie_name: str):
    movie_id2 = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id2)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

- toy story와 비슷한 영화 찾아보기

- 다수의 애니메이션 영화가 나오는 것을 확인할 수 있었다.

In [ ]:
df = pd.DataFrame(get_similar_movie('toy story (1995)'))
df.columns = ['title']
df

,title
0,toy story (1995)
1,toy story 2 (1999)
2,aladdin (1992)
3,"bug's life, a (1998)"
4,babe (1995)
5,groundhog day (1993)
6,"lion king, the (1994)"
7,beauty and the beast (1991)
8,pleasantville (1998)
9,there's something about mary (1998)


- One Flew Over the Cuckoo's Nest와 비슷한 영화를 확인해 보았다.

- One Flew Over the Cuckoo's Nest는 범죄, 교도소, 정신병원 등을 배경으로 하는 코미디 드라마 장르의 영화이다.

- 정말 흥미롭게도 유사하다고 나온 영화들이 범죄, 감옥이 연관된 영화가 많다!

In [ ]:
df2 = pd.DataFrame(get_similar_movie('one flew over the cuckoo\'s nest (1975)'))
df2.columns = ['title']
df2

,title
0,one flew over the cuckoo's nest (1975)
1,apocalypse now (1979)
2,"shawshank redemption, the (1994)"
3,schindler's list (1993)
4,midnight cowboy (1969)
5,"sunchaser, the (1996)"
6,amadeus (1984)
7,"adventures of milo and otis, the (1986)"
8,dog park (1998)
9,papillon (1973)


## 영화 추천 받아보기

In [ ]:
user = user_to_idx['6100']
# recommend에서는 user*item CSR Matrix를 받습니다.
# filter_already_liked_items: 유저가 이미 본 영화는 제외
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(120, 0.6082065),
 (64, 0.5409119),
 (172, 0.46695912),
 (44, 0.4361921),
 (200, 0.38118833),
 (5, 0.35645068),
 (243, 0.31230456),
 (13, 0.29915622),
 (680, 0.26902986),
 (651, 0.260333),
 (15, 0.2542924),
 (189, 0.24874298),
 (7, 0.23267923),
 (488, 0.22632773),
 (527, 0.22330981),
 (26, 0.21482837),
 (637, 0.21412359),
 (92, 0.21081987),
 (648, 0.20890892),
 (87, 0.20641075)]

In [ ]:
df3 = pd.DataFrame([idx_to_movie[i[0]] for i in movie_recommended])
df3.columns = ['title']
df3

,title
0,raiders of the lost ark (1981)
1,star wars: episode vi - return of the jedi (1983)
2,indiana jones and the last crusade (1989)
3,star wars: episode iv - a new hope (1977)
4,"terminator, the (1984)"
5,"princess bride, the (1987)"
6,ghostbusters (1984)
7,ferris bueller's day off (1986)
8,blade runner (1982)
9,aliens (1986)


- 추천된 영화들이 사전에 선호한다고 선택한 5개의 영화들에 어느 정도 영향을 받았는지 확인<br>
 (사전 선택 영화의 기여도 확인)

- 기여도를 잘 나타내는지 보고 싶어서 의도적으로 star wars4로 확인해보았다.

- star wars5가 0.473으로 가장 높은 기여도를 나타내는 것으로 보아 잘 작동하는 것으로 판단 된다.

In [ ]:
#  explain 메소드를 사용하면 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인
star_wars6 =movie_to_idx['star wars: episode vi - return of the jedi (1983)']
explain = als_model.explain(user, csr_data, itemid=star_wars6)

In [ ]:
df4 = pd.DataFrame([(idx_to_movie[i[0]], i[1]) for i in explain[1]])
df4.columns = ['title', 'Contribution']
df4

,title,Contribution
0,star wars: episode v - the empire strikes back...,0.473476
1,back to the future (1985),0.077007
2,who framed roger rabbit? (1988),0.015665
3,die hard (1988),0.001597
4,stand by me (1986),-0.032957


- raiders of the lost ark로 기여도를 한 번 더 확인해보았다.

- 이번 영화도 star wars가 0.278로 가장 높은 기여도를 보였다.

- 두 영화 모두 제작에 조지 루커스가 관여 되어있고, 액션 영화인 것을 생각하면 기여도가 잘 나타난 것 같다.

In [86]:
raiders =movie_to_idx['raiders of the lost ark (1981)']
explain2 = als_model.explain(user, csr_data, itemid=raiders)

In [87]:
df5 = pd.DataFrame([(idx_to_movie[i[0]], i[1]) for i in explain2[1]])
df5.columns = ['title', 'Contribution']
df5

,title,Contribution
0,star wars: episode v - the empire strikes back...,0.278401
1,die hard (1988),0.153315
2,stand by me (1986),0.076373
3,back to the future (1985),0.048109
4,who framed roger rabbit? (1988),0.042334


## 회고

- 이번 프로젝트는 조금  흥미로운 점이 많았던 것 같다.

- 아무래도 일상 생활에서 어렵지 않게 이용하고 있었던 추천 시스템을 다뤘기 때문이었던 것 같다.

- 최근에는 넷플릭스나 멜론과 같은 플랫폼 이외에도 에이블리나 쿠팡 등의 쇼핑몰에서도 추천 시스템이 많이 활용되고 있고 나 역시 잘 사용하고 있다.

- 하지만 사용을 하면서도 이 서비스가 어떻게 제공 되고 있는 것인지,, 궁금하면서도 조금 무섭다는 생각이 들었었다.

- 직접 추천 시스템 코드를 작성해보니 궁금했던 부분이 많이 해소 되었다.

- 그렇지만 아쉬운 점은 필수 컬럼이 아니라고 제외 했던 영화 장르를 활용해보지 못한 것이다.. 프로젝트 제출 기한만 아니었으면 활용해보고 싶은 욕망이....ㅠ 

- 나중에 조금 한가할 때(? 과연 올까..) 다시 다뤄보고 싶다.
